In [2]:
from elliot.run import run_experiment
import zipfile
import io
import requests
import os
import pandas as pd
import numpy as np

import pandas as pd
pd.options.display.max_rows = 100
import numpy as np
import os
from lenskit import crossfold as xf
from data_generation import generate_data
from itertools import chain
from lenskit import util, batch, topn
import modelling_mf

In [13]:
filename = 'NGCF_seed=42_e=50_bs=256_lr=0$0001_factors=64_l_w=0$1_weight_size=64_node_dropout=_message_dropout=0$1_n_fold=5_it=50.tsv'

In [5]:
data="fairbook"
# user-item interactions
fairbook_ratings = pd.read_csv("elliot/data/fairbook/"+data+"_events.csv")

In [7]:
user_col = "user" # the name of the column that includes the users
item_col = "item" # the name of the column that includes the items
predict_col="rating" # the name of the column that includes the interaction

In [8]:
data_strategies = [
                   'popularity_good',
                   ]

In [9]:
location = 'elliot/results/pop_good1/recs/'

In [10]:
ratings = generate_data(strategy = data_strategies[0],
                            copying_dataset = fairbook_ratings,
                            user_perc = 0.2)

In [11]:
all_items=set(ratings.item.unique())

In [14]:
pg_metrics = []
train_df = pd.read_csv('elliot/data/popularity_good/popularity_good_fold_1_train.csv')
test_df = pd.read_csv('elliot/data/popularity_good/popularity_good_fold_1_test.csv')
result_file = location+filename
print(result_file)
test_users = test_df.user.unique()
result_df = pd.read_csv(result_file, sep='\t', header=None)

    # result_df.columns = ['user','item','rating']
    # test_result = result_df[result_df.user.isin(test_users)].reset_index(drop=True)
    
    # end = len(test_result)
    # # Create an iterable of indices to keep
    # indices_to_keep = list(chain.from_iterable(range(t, t + keep_interval) for t in range(start, end, keep_interval + skip_interval)))
    # filtered_test_result = test_result.iloc[indices_to_keep].reset_index(drop=True) # keep top 10
    # # filtered_test_result = test_result.copy()
    # recs_grouped = filtered_test_result.groupby([user_col])[item_col].apply(list)
    # pop_bias= modelling_mf.calculate_pop_bias_per_item(all_items, item_col, user_col, predict_col, train_df, recs=filtered_test_result)
    # GAP_vs_GAP = modelling_mf.calculate_ave_pop_per_user(test_users, item_col, user_col, pop_bias, train_df, recs_grouped)
    # pop_corr = modelling_mf.calculate_pop_correlation(pop_bias)
    # precision, recall, ndcg = modelling_mf.calculate_topn_metrics(filtered_test_result,test_df)
    # AggDiv = modelling_mf.evaluate_item_coverage(pop_bias["recommendation"].values)
    # ARP, ave_PL, ACLT = modelling_mf.calculate_all_pb_metrics(pop_bias, test_users, item_col, user_col, train_df, recs_grouped, filtered_test_result)
    # metrics_dict_pg = {"pop_corr":pop_corr, "RMSE":0, 'NDCG':ndcg,"ARP":ARP, "ave_PL": ave_PL, "ACLT": ACLT, "AggDiv": AggDiv}
    # pg_metrics.append(metrics_dict_pg)
    # pop_biases = [pop_bias]
    # modelling_mf.plot_results(pop_biases.copy(), 
    #          GAP_vs_GAP.copy(), 'CornacNGCF',
    #          0, 
    #          precision, 
    #          recall,
    #          ndcg,
    #          0,
    #          0,
    #          cv=False, 
    #          n=10, 
    #          args='fold'+str(i+1), data_strategy=data_strategies[j], save_plot=True)

elliot/results/pop_good1/recs/NGCF_seed=42_e=50_bs=256_lr=0$0001_factors=64_l_w=0$1_weight_size=64_node_dropout=_message_dropout=0$1_n_fold=5_it=50.tsv


In [20]:
len(result_df.groupby(0).mean())

6358